In [123]:
import re
tradeDay=''
send_headers = {
        'Host': 'query.sse.com.cn',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:37.0) Gecko/20100101 Firefox/37.0',
        'Accept': '*/*',
        'Accept-Language': 'zh-CN,zh;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate',
        'Referer': 'http://www.sse.com.cn/disclosure/diclosure/public/',
        'Connection': 'keep-alive'
    }
url='http://query.sse.com.cn/infodisplay/showTradePublicFile.do?jsonCallBack=jQuery172023210379532913938_1430627585124&dateTx=2015-04-29&random=0.48195114223841695&_=1430627617454'
req = urllib2.Request(url,headers=send_headers)
response = urllib2.urlopen(req)
html = response.read()
jsonStr=demjson.decode(html[len('jQuery172023210379532913938_1430627585124('):-1])
lines=jsonStr['fileContents']
def loopLineFun(lines):
    for line in lines:
        yield line.encode('utf8')
loopline=loopLineFun(lines)
class LHBItem():
    pass
dictlist = {}
r1 = re.compile(ur'\s+\(\d\)\s+(\d+)\s+([\u4e00-\u9fa5]+)\s+((-?\d+)(\.\d+)?)%\s+(\d+)\s+((-?\d+)(\.\d+)?)')
#r1 = re.compile(ur'\s+\(\d\)')

def readDep(loop,code):
    state='buy'
    rdep = re.compile(ur'\s+\(\d\)')
    rout=re.compile(ur'^\s+$')
    for tmp in loop:
        print tmp
        if tmp.find('买入营业部名称')>=0:
            state='buy'
            continue
        if tmp.find('卖出营业部名称')>=0:
            state='sell'
            continue
        if rout.match(tmp.decode('utf8')) and state=='sell':
            print '跳出'
            return
        if rdep.match(tmp.decode('utf8')):
            dep=re.split('\s+',tmp)
            depName=dep[2]
            tradeAmount=dep[3]
            print 'depName ' + depName


r2=re.compile(ur'\s+[\u4e00-\u9fa5]+:\s(\d+)\s+[\u4e00-\u9fa5]+:\s[\u4e00-\u9fa5]+')
def readA7(loop):
    for tmp in loop:
        mat=r1.match(tmp.decode('utf8'))
        if mat:
            lbhItem =LHBItem()
            lbhItem.symbol= mat.group(1)
            lbhItem.stockName= mat.group(2)
            lbhItem.zhengdie= mat.group(3)
            lbhItem.vol=mat.group(6)
            lbhItem.amount= mat.group(7)
            dictlist[lbhItem.symbol]=lbhItem
            continue
        #dep
        mat2=r2.match(tmp.decode('utf8'))
        if mat2:
            print '*************************'
            readDep(loop,mat2.group(1))
        if tmp.find('二、')>=0:
            return


for tmp in loopline:
    print tmp
    if tmp.find('交易日期')>=0:
        tradeDay=tmp[13:]
        print tradeDay
    if tmp.find('偏离值达到7%')>=0:
        tmp=readA7(loopline)
        print tmp;
        break
    if tmp.find('二、')>=0:
        print '-------'
for k in dictlist:
    print k


            上海证券交易所每日交易信息

交易日期:2015年04月29日
2015年04月29日


一、有价格涨跌幅限制的日收盘价格涨幅偏离值达到7%的前三只证券:
*************************
      ------------------------------------------------------------------------------
      买入营业部名称:                                              累计买入金额(元):
  (1) 招商证券股份有限公司北京知春东里证券营业部                           207500.00
depName 招商证券股份有限公司北京知春东里证券营业部

      卖出营业部名称:                                              累计卖出金额(元):
  (1) 中信证券（浙江）有限责任公司宁波中山东路证券营业部                    12500.00
depName 中信证券（浙江）有限责任公司宁波中山东路证券营业部
  (2) 渤海证券股份有限公司上海定西路证券营业部                              12500.00
depName 渤海证券股份有限公司上海定西路证券营业部
  (3) 中信证券股份有限公司上海石化证券营业部                                12500.00
depName 中信证券股份有限公司上海石化证券营业部
  (4) 长城证券有限责任公司北京中关村大街证券营业部                          12500.00
depName 长城证券有限责任公司北京中关村大街证券营业部
  (5) 申银万国证券股份有限公司上海三林路营业部                              12500.00
depName 申银万国证券股份有限公司上海三林路营业部

      证券代码: 600311                                            证券简称: 荣华实业
      -----------